In [8]:
## import modules

import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from ipywidgets import interact, Layout
import ipywidgets as widgets
from bokeh.io import push_notebook, show, output_notebook
output_notebook()
from bokeh.plotting import figure
from bokeh.layouts import row, widgetbox, column
from bokeh.models.widgets import PreText
from itertools import product as carpro
from collections import OrderedDict as od
from pickle import dump, load

Loading BokehJS ...

In [9]:
## func for trading

def get_trading_data(df_res, datepair, days_forw = 5):
    dfslice = df_res[(datepair[0] < df_res['date']) & 
                (df_res['date'] < datepair[1])]
    
    start_price = dfslice.iloc[0]['price']

    def trade(up_pred_name = 'up_pred'):
        money = 100.
        shares = 0.
        price = None
        sell_date = None
        up_pred = False
        hold = False
        invest_frac = 1./1
        assets_series = []
        assets_passive_series = []

        for ix, today in dfslice.iterrows():
            price = today['price']
            up_pred = today[up_pred_name]
            date = today['t']

            if up_pred: # upward forecast
                if sell_date == None or \
                    (sell_date != None and not hold):
                    # either nothing's been bought
                    # or no negative forecast's been seen
                    # so buy
                    shares += money*invest_frac / price
                    money -= money*invest_frac
                    sell_date = date + days_forw - 1
                    hold = False

            else: # downward forecast, hold
                hold = True

            if date == sell_date: # sell
                money += shares*price
                shares = 0
                sell_date = None

            if up_pred_name == 'up_pred':
                assets_series.append(money + shares*price)
                assets_passive_series.append(
                    (1 + (price - start_price)/start_price)*100)

        return money, shares, price, np.array(assets_series), \
            np.array(assets_passive_series)
        

    money, shares, price, assets_series, assets_passive_series \
        = trade()
    beat_market_bools = (assets_series > assets_passive_series)*1
    beat_market_perc = sum(beat_market_bools)/len(beat_market_bools)*100
    assets = assets_series[-1]
    assets_passive = assets_passive_series[-1]
    share_value = shares*price

    money_ideal, shares_ideal, price_ideal, _, _ = trade(up_pred_name = 'up')
    assets_ideal = money_ideal + shares_ideal*price_ideal

    perf = (assets - 100)
    perf_wrt_market = (assets - assets_passive)/assets_passive*100
    years = (df_res.iloc[-1]['t'] - df_res.iloc[0]['t'])/261
    
    return {'assets_series': assets_series,
            'assets': assets,
            'assets_passive_series': assets_passive_series,
            'assets_passive': assets_passive,
            'money': money,
            'shares': shares,
            'price': price,
            'beat_market_perc': beat_market_perc,
            'perf': perf,
            'perf_wrt_market': perf_wrt_market,
            'years': years,
           }

In [12]:
## visualize

df_res = pd.read_csv('df_res.csv')

dates = list(df_res['date'].values)
datepairdefault = (dates[0], dates[-1])

data = get_trading_data(df_res, datepairdefault)

days = range(len(data['assets_series']))
asset_series = data['assets_series']
asset_passive_series = data['assets_passive_series']

p = figure(title='Total Asset Value', plot_height=300, plot_width=600, 
           x_axis_label = "Day")
r_pass = p.line(days, asset_passive_series, color="dodgerblue", line_width=1, 
                 legend = "passive")
r_strat = p.line(days, asset_series, color="orange", line_width=1, 
                legend = "strategy")
p.legend.location = "top_left"

def get_info(data, datepair):
    return "Period: " + datepair[0] + " to " + datepair[1] + "\n"  + \
        "--------------------------------" + "\n" + \
        "Years = " + "{0:.2f}".format(data['years']) + "\n" + \
        "Profit = " + "{0:.2f}".format(data['perf']) + "%" + \
        " (Market rise = " + "{0:.2f}".format(data['assets_passive'] - 100
                                         ) + "%)\n" + \
        "Performance w.r.t market = " + \
        "{0:.2f}".format(data['perf_wrt_market']) + "%\n" + \
        "Beat the market: " + "{0:.2f}".format(data['beat_market_perc']) + \
        "% of the time\n"

info_box = PreText(text=get_info(data, datepairdefault),
width=300)

empty_space = PreText(text = "", width = 50)

layout = row(p, empty_space, info_box)
    
show(layout, notebook_handle=True)